In [1]:
# %pip install peft
# %pip install gradio

import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from peft import PeftModel, PeftConfig 
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer, TextStreamer 
import torch 
from threading import Thread 
import gradio as gr 

In [2]:
peft_model_id = "smangrul/mistral-dpo" # "smangrul/mistral-sft" 
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, load_in_4bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=8)
model = PeftModel.from_pretrained(model, peft_model_id)
model.eval()

ValueError: Can't find 'adapter_config.json' at 'smangrul/mistral-dpo'

In [ ]:
class ChatCompletion:
    
    def __init__(self, model, tokenizer, system_prompt=None):
        self.model = model 
        self.tokenizer = tokenizer 
        self.streamer = TextIteratorStreamer(self.tokenizer, skip_prompt=True)
        self.print_streamer = TextStreamer(self.tokenizer, skip_prompt=True)
        # set the model in inference mode 
        self.model.eval()
        self.system_prompt = system_prompt 
        
    def get_completion(self, prompt, system_prompt=None, message_history=None, max_new_tokens=512, temperature=0.0):
        if temperature < 1e-2:
            temperature = 1e-2 
        messages = [] 
        if message_history is not None:
            messages.extend(message_history)
        elif system_prompt or self.system_prompt:
            system_prompt = system_prompt or self.system_prompt 
            messages.append({"role": "system", "content":system_prompt})
        messages.append({"role": "user", "content": prompt})
        chat_prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        inputs = self.tokenizer(chat_prompt, return_tensors="pt", add_special_tokens=False)
        # Run the generation in a separate thread, so that we can fetch the generated text in a non-blocking way.
        generation_kwargs = dict(max_new_tokens=max_new_tokens,
                                temperature=temperature,
                                top_p=0.95,
                                do_sample=True,
                                repetition_penalty=1.1)
        generated_text = self.model.generate(**inputs, streamer=self.print_streamer, **generation_kwargs)
        return generated_text